# Deep Feature Selection
## User Guide on nonlinear example
In this notebook, we will demonstrate how to implement our method on our nonlinear example from the paper.

In [1]:
import sys
sys.path.append("../../src")
from time import clock
import numpy as np
import math
import pandas as pd 
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import grad
from torch.nn.parameter import Parameter
from utils import data_load_n, measure, accuracy
from models import Net_nonlinear
from dfs import DFS_epoch

### Data Preparation
We will load our data in the following chunk. The data, both covariates and response, need to be load as `pytorch` `Tensor` objects to be fed into the DFS algorithm. The function `data_load_n` will read in dataset and split it into training and test set so that both sets have same number of positive and negative samples.

In [2]:
# load and prepare datasets
dirc = "../../data/nonlinear/p_500_N_600_s_4/"
k = 0 # dataset number from 0 to 29
X, Y, X_test, Y_test = data_load_n(k, directory=dirc)
N, p = X.shape
print("The covariates is of type:", type(X))
print("The response is of type:", type(Y))
print("The dimension of training set:", X.shape)
print("    The number of positive sample:", len(np.where(Y==1)[0]))
print("    The number of negative sample:", len(np.where(Y==0)[0]))
print("The dimension of test set:", X.shape)
print("    The number of positive sample:", len(np.where(Y_test==1)[0]))
print("    The number of negative sample:", len(np.where(Y_test==0)[0]))

The covariates is of type: <class 'torch.Tensor'>
The response is of type: <class 'torch.Tensor'>
The dimension of training set: torch.Size([300, 500])
    The number of positive sample: 150
    The number of negative sample: 150
The dimension of test set: torch.Size([300, 500])
    The number of positive sample: 150
    The number of negative sample: 150


### DFS with fixed hyper-parameters
In this section, we demonstrate how to run DFS with one given set of hyper-parameters. The hyper-parameters includes:
* `s`, the number of variables to be selected;
* `c`, the tunning parameters to control the magnitude of $\lambda_1$ and $\lambda_2$;
* `epochs`, the number of DFS iterations to be run;
* `n_hidden1` & `n_hidden2`, the number of neurons in the fully connect neural networks;
* `learning_rate`, the learning rate for optimizer;
* `Ts` & `step`, the parameters to control the optimization on given support

Among the above hyper-parameters, `s` is the most important parameters, and the selection of $s$ will be demonstrated in next sections. `c` can be selection through a sequence of candidates that returns the smallest loss function. Others mostly are meant to help the convergence of the optimization steps.

In [3]:
# specify hyper-paramters
s = 4
c = 1
epochs = 10
n_hidden1 = 50
n_hidden2 = 10
learning_rate = 0.05
Ts = 25 # To avoid long time waiting, this parameter has been shorten
step = 5
# Define Model
torch.manual_seed(1) # set seed
# Define a model with pre-specified structure and hyper parameters
model = Net_nonlinear(n_feature=p, n_hidden1=n_hidden1, n_hidden2=n_hidden2, n_output=2)
# Define another model to save the current best model based on loss function value
# The purpose is to prevent divergence of the training due to large learning rate or other reason
best_model = Net_nonlinear(n_feature=p, n_hidden1=n_hidden1, n_hidden2=n_hidden2, n_output=2)
# Define optimizers for the optimization with given support
# optimizer to separately optimize the hidden layers and selection layers
# the selection layer will be optimized on given support only.
# the optimzation of hidden layers and selection layer will take turn in iterations
optimizer = torch.optim.Adam(list(model.parameters()), lr=learning_rate, weight_decay=0.0025*c)
optimizer0 = torch.optim.Adam(model.hidden0.parameters(), lr=learning_rate, weight_decay=0.0005*c)
# Define loss function
lf = torch.nn.CrossEntropyLoss()
# Allocated some objects to keep track of changes over iterations
hist = []
SUPP = []
LOSSES = []
supp_x = list(range(p)) # initial support
SUPP.append(supp_x)
### DFS algorithm
start = clock()
for i in range(epochs):
    # One DFS epoch
    model, supp_x, LOSS = DFS_epoch(model, s, supp_x, X, Y, lf, optimizer0, optimizer, Ts, step)
    LOSSES = LOSSES + LOSS
    supp_x.sort()
    # Save current loss function value and support
    hist.append(lf(model(X), Y).data.numpy().tolist())
    SUPP.append(supp_x)
    # Prevent divergence of optimization over support, save the current best model
    if hist[-1] == min(hist):
        best_model.load_state_dict(model.state_dict())
        best_supp = supp_x
    # Early stop criteria
    if len(SUPP[-1]) == len(SUPP[-2]) and (SUPP[-1] == SUPP[-2]).all():
        break

end = clock()
print("DFS algorithm finishs in", end-start, "seconds")

Final Support:  [2 0 3 1]
Final Support:  [2 0 3 1]
DFS algorithm finishs in 118.80000000000001 seconds


In the following chunk, we will demonstrate the results from the DFS algorithm. Note that the training and test error is not the final results, as we recommend two-step procedure.

In [9]:
# Metric calculation
err_train_1 = 1-accuracy(best_model, X, Y)
err_test_1 = 1-accuracy(best_model, X_test, Y_test)
print("The support selected is:", best_supp)
print("The index of non-zero coefficents on selection layer:", 
      np.where(best_model.hidden0.weight != 0)[0])
print("The training error is:", err_train_1)
print("The test error is:", err_test_1)

The support selected is: [0 1 2 3]
The index of non-zero coefficents on selection layer: [0 1 2 3]
The training error is: 0.010000000000000009
The test error is: 0.043333333333333335


In the following chunk, we will perform a two-step procedure to train the `best_model` on the given support. Two-step procedure is used for two reasons, to get better predictive performance and to get better estimation of `bic` which is important in selection of optimal $s$.

As we demonstrated on the above chunk, the selection layer of `best_model` has non-zero coefficients on given support. In the second step, we treat `best_model` as our initial model and update parameters only in hidden layer.

In [11]:
_optimizer = torch.optim.Adam(list(best_model.parameters())[1:], lr=0.01, weight_decay=0.0025)
for _ in range(100):
    out = best_model(X)
    loss = lf(out, Y)
    _optimizer.zero_grad()
    loss.backward()
    _optimizer.step()

bic = (loss.data.numpy().tolist())*N*2. + s*np.log(N)

### Selection of $s$
In this section, we demonstrate the procedure of 